# Exercise 4 

## Importing Libraries

In [75]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from itertools import permutations
from scipy.spatial import distance

## Cleaning DataFrame

In [76]:
wines = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols',
              'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315',\
              'Proline']

# I create the columns name based on the file wine.names

In [77]:
f=pd.read_csv('C:\\Users\\Daniele\\Desktop\\HW 4\\wine.data',names=wines)

## Dropping the NaN values

In [78]:
pd.DataFrame.drop(f,['Class'],axis=1,inplace=True)

# I remove the columns class, and I drop the empty space (NaN) replace with 0. 

In [79]:
f.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


## Standardize to avoid a big influence from variable like Proline and Magnesium

In [80]:
scaler = StandardScaler()

# I standardize to avoid the influence of big variable.

In [81]:
scaled_array = scaler.fit_transform(f)

# I apply the standardization for all the columns of f(dataframe) with the function fit

In [82]:
scaled_data= pd.DataFrame( scaled_array, columns = f.columns )

In [83]:
scaled_data.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874


## Defining the Kmeans

In [84]:
def Kmeans(array,k):
    
    def initialize_centroids(array,k):
        initial_centroids = np.random.permutation(array.shape[0])[:k]
        centroids = array[initial_centroids]
        return centroids
    
    # I define the initialize centroids that consists of making a random list of centroids.
    
    lis_initial_centroids = initialize_centroids(array,k)
    
    def assign_centroids(array,centroids):
        lis_clustering=[]
        for e in array:
            d_e = []
            for i in range(k):
                d = distance.euclidean(e,centroids[i])
                d_e.append(d)
            min_distance=min(d_e)
            lis_clustering.append((d_e.index(min_distance)+1))
        lis_clustering= np.array(lis_clustering)
        clusters_to_append = lis_clustering.reshape(178,1)   
        new_array = np.append(array,clusters_to_append,axis=1)
        return new_array
    
    # I assign for each data the nearest centroid. Create the clusters. 
        
    new_array = assign_centroids(scaled_array,lis_initial_centroids)
    
    def new_centroids(new_array):
        lis_centroids = []
        centroids = []
        for i in range(k):
            lis_centroids.append([])
        for j in range(178):
            for i in range(k):
                if float(i+1) == new_array[j][13]:
                    lis_centroids[i].append(new_array[j])
        for i in range(k):
            lis_centroid = lis_centroids[i]
            lis_centroid = np.delete(lis_centroid,13,axis=1)
            new_centroid = np.mean(lis_centroid,axis=0)
            centroids.append(new_centroid)
        return centroids
   
    # I calculate the mean for each clusters. Make the new centroids.
    
    centroids = new_centroids(new_array)
    
    final_array = assign_centroids(scaled_array,centroids)
    
    def update_centroids(initial_centroids):
        centroids_old = np.zeros((k,12))
        centroids = initial_centroids
        numItem = 0
        while ([list(elem) for elem in centroids_old] != [list(elem) for elem in centroids]):
            numItem += 1
            centroids_old = centroids.copy()
            new_array = assign_centroids(scaled_array,centroids_old)
            centroids = new_centroids(new_array)
        return numItem,centroids,centroids_old
    
    # Update_centroids,needs to repeat the reassignment of new_centroids until the old_centroids = new_centorids.
    
    numItem = update_centroids(lis_initial_centroids)[0]
    
    # numItem is the nums of iterations that happens until old_centroids = new_centorids
    
    centroids = update_centroids(lis_initial_centroids)[1]
    
    final_array = assign_centroids(scaled_array,centroids)
    
    def clustering(array):
        lis_clusters = []
        clusters = []
        for i in range(k):
            lis_clusters.append([])
        for j in range(178):
            for i in range(k):
                if float(i+1) == array[j][13]:
                    lis_clusters[i].append(array[j])
        for i in range(k):
            cluster = lis_clusters[i]
            cluster = np.delete(cluster,13,axis=1)
            clusters.append(cluster)
        return clusters
    
    # Reassign data to the new_centroids.
    
    clusters = clustering(final_array)
    
    def dissimilarity(clusters,centroids):
        Dist = []
        for c in range (len(clusters)):
            Dist_cluster = ([])
            for e in clusters[c]:
                Dist_cluster += [(distance.euclidean(e,centroids[c]))**2]
            Dist += [Dist_cluster]
        final_dist = []
        for e in Dist:
            e = (sum(e))
            final_dist += [e]
        totDist = (sum(final_dist))
        return totDist
    
    # Calculate dissimilarity, the total distance from each data and its own centroids. 
        
    totDist = dissimilarity(clusters,centroids)
    
    return totDist,clusters,centroids,lis_initial_centroids

In [85]:
results = Kmeans(scaled_array,3)

## Defining the Best Kmeans 

In [86]:
def bestKmeans(rep):
    best = Kmeans(scaled_array,3)
    minDissimilarity = best[0]
    for trial in range(rep):
        result = Kmeans(scaled_array,3)
        currDissimilarity = result[0]
        if currDissimilarity < minDissimilarity:
            best = result
            minDissimilarity = currDissimilarity       
    return best[0],best[2],best[3]
    
# best Kmeans consists on the repetition of K-means and the results is the best Kmeans otherwise the Kmeans with smaller dissimilarity.    

## Defining the Worst Kmeans

In [87]:
def worstKmeans(rep):
    worst = Kmeans(scaled_array,3)
    minDissimilarity = worst[0]
    for trial in range(rep):
        result = Kmeans(scaled_array,3)
        currDissimilarity = result[0]
        if currDissimilarity > minDissimilarity:
            worst = result
            minDissimilarity = currDissimilarity       
    return worst[0],worst[2],worst[3]

# worst Kmeans consists on the repetition of K-means and the results is the worst Kmeans otherwise the Kmeans with bigger dissimilarity.

## Results

In [88]:
best = bestKmeans(50)

In [89]:
worst = worstKmeans(50)

In [90]:
best

(1277.9284888446423,
 [array([ 0.83523208, -0.30380968,  0.36470604, -0.61019129,  0.5775868 ,
          0.88523736,  0.97781956, -0.56208965,  0.58028658,  0.17106348,
          0.47398365,  0.77924711,  1.12518529]),
  array([-0.92607185, -0.39404154, -0.49451676,  0.17060184, -0.49171185,
         -0.07598265,  0.02081257, -0.03353357,  0.0582655 , -0.90191402,
          0.46180361,  0.27076419, -0.75384618]),
  array([ 0.16490746,  0.87154706,  0.18689833,  0.52436746, -0.07547277,
         -0.97933029, -1.21524764,  0.72606354, -0.77970639,  0.94153874,
         -1.16478865, -1.29241163, -0.40708796])],
 array([[ 1.02450655, -0.61610959,  0.8534605 , -0.68913719, -0.40313502,
          0.24818135,  0.96454129, -1.14303148,  1.22488398,  0.23441405,
          1.23964502,  1.07107602,  1.64990767],
        [ 1.1109751 , -0.58917969, -0.90110314, -1.04947918,  0.08835836,
          1.28969686,  1.36612798, -1.22360954,  0.96207136,  0.45069904,
         -0.20817676,  1.01457831,  0.7

In [91]:
worst

(1617.3744851527877,
 [array([ 0.04197899, -0.38224053, -0.10922044, -0.29717226,  0.08897972,
          0.5428402 ,  0.60204711, -0.48200615,  0.41194796, -0.31261472,
          0.50738654,  0.61509448,  0.26067933]),
  array([-0.11095684,  0.70445264,  0.16068636,  0.49246043, -0.14568611,
         -0.99426144, -1.05359118,  0.8228174 , -0.78917844,  0.46024304,
         -0.85611008, -1.07000793, -0.4567613 ]),
  array([ 1.12332775, -0.59366801,  1.10933436,  1.27772951, -0.43824169,
          0.64876424, -0.82753931,  1.31459937,  1.58406123,  3.16507569,
         -1.69987192, -1.0475882 , -0.34040091])],
 array([[ 0.59216382, -0.47248348,  0.15894572,  0.30180329,  0.01814502,
          0.64876424,  0.95450162, -0.82071924,  0.47148781,  0.01812906,
          0.36217728,  1.2123203 ,  0.55125733],
        [ 0.49334262,  1.41260912,  0.41481959,  1.05251577,  0.1585717 ,
         -0.79333415, -1.28434417,  0.5491078 , -0.31695005,  0.96978302,
         -1.12951789, -1.48544548,  0.0

### Comparing the results the cost is higher where the initials centroids were bad chosen. 